In [1]:
import torch.optim as optim
import torch

In [47]:
# G_input = torch.tensor([2.,2.,2.])
# G_param = torch.tensor([1.,2.,3.],requires_grad=True)
# G_out = (G_param*G_input).detach()# detach()返回一个指向同一内存的新变量，唯一的不同点是返回的这个东西的require_grad = false，反向传播到此处戛然而止
# D_param = torch.tensor([2.,2.,2.],requires_grad=True)
# D_out = (G_out * D_param).sum()
# D_out.backward()
# print(D_param.grad,G_param.grad)

In [48]:

G_input = torch.tensor([1.])
G_param = torch.tensor([1.],requires_grad=True)
optimizer_G = optim.Adam([G_param], lr = 0.0001)
D_param = torch.tensor([2.],requires_grad=True)
optimizer_D = optim.Adam([D_param], lr = 0.0001)

In [49]:
# 得出G的输出，并且经过某种损失函数更新G，此法不通，最终决定CE和adv分开训练
# G_out = (G_param*G_input).detach()

In [50]:
# 得到G的输出
G_out = (G_param*G_input)
D_real = (G_out.detach() * D_param).sum()#D要尽可能的把输出最小化，所以直接对输出梯度下降

#D先更新
D_real.backward()
print("对D反传后 | ","D的梯度：",D_param.grad.item()," |D的参数大小：",D_param.item()
      ," |G的梯度：",G_param.grad," |G的参数大小：",G_param.item())
optimizer_D.step() #G是把detach掉的东西给D的，无梯度，D的参数都有梯度
print("对D梯度下降后 | ","D的梯度：",D_param.grad.item()," |D的参数大小：",D_param.item()
      ," |G的梯度：",G_param.grad," |G的参数大小：",G_param.item())
optimizer_D.zero_grad()
print("对D梯度清空梯度后 | ","D的梯度：",D_param.grad.item()," |D的参数大小：",D_param.item()
      ," |G的梯度：",G_param.grad," |G的参数大小：",G_param.item())


对D反传后 |  D的梯度： 1.0  |D的参数大小： 2.0  |G的梯度： None  |G的参数大小： 1.0
对D梯度下降后 |  D的梯度： 1.0  |D的参数大小： 1.999899983406067  |G的梯度： None  |G的参数大小： 1.0
对D梯度清空梯度后 |  D的梯度： 0.0  |D的参数大小： 1.999899983406067  |G的梯度： None  |G的参数大小： 1.0


In [51]:
#出现了bug，再D梯度下降之后，对D的参数进行了inplace操作，无法传梯度回去了，所以我依次更新可以吗

In [52]:
#G再更新
D_fake = -(G_out * D_param).sum()#G要尽可能把输出最大化，所以对输出的负值梯度下降
D_fake.backward()
print("对D梯 | ","D的梯度：",D_param.grad.item()," |D的参数大小：",D_param.item()
      ," |G的梯度：",G_param.grad," |G的参数大小：",G_param.item())
optimizer_G.step() #只有G有梯度
print("对D梯 | ","D的梯度：",D_param.grad.item()," |D的参数大小：",D_param.item()
      ," |G的梯度：",G_param.grad," |G的参数大小：",G_param.item())
optimizer_D.zero_grad()
print("对D梯度清空梯度后 | ","D的梯度：",D_param.grad.item()," |D的参数大小：",D_param.item()
      ," |G的梯度：",G_param.grad," |G的参数大小：",G_param.item())
optimizer_G.zero_grad()
print("对D梯度清空梯度后 | ","D的梯度：",D_param.grad.item()," |D的参数大小：",D_param.item()
      ," |G的梯度：",G_param.grad," |G的参数大小：",G_param.item())

对D梯 |  D的梯度： -1.0  |D的参数大小： 1.999899983406067  |G的梯度： tensor([-1.9999])  |G的参数大小： 1.0
对D梯 |  D的梯度： -1.0  |D的参数大小： 1.999899983406067  |G的梯度： tensor([-1.9999])  |G的参数大小： 1.000100016593933
对D梯度清空梯度后 |  D的梯度： 0.0  |D的参数大小： 1.999899983406067  |G的梯度： tensor([-1.9999])  |G的参数大小： 1.000100016593933
对D梯度清空梯度后 |  D的梯度： 0.0  |D的参数大小： 1.999899983406067  |G的梯度： tensor([0.])  |G的参数大小： 1.000100016593933
